In [1]:
import sys
# Python 3.7 is required
assert sys.version_info >= (3,7)

import cv2 as cv
import numpy as np

# Make sure that optimization is enabled
if not cv.useOptimized():
    cv.setUseOptimized(True)

cv.useOptimized()

True

In [2]:
#Question 1

def crop_grid(img, num_horizontal_grid, num_vertical_grid, line_color):
    
    img_copy = img.copy()
    height, width = img.shape[:2]
    
    # M and N are basically number of pixels per patch
    M, N = int(height/num_horizontal_grid), int(width/num_vertical_grid)
    
    
    x1, y1 = 0, 0

    for y in range(0, height, M):
        for x in range(0, width, N):
            #if (height - y) < M or (width - x) < N:
                #break
            
            y1 = y + M    # lower right coordinate that will be used to construct rectangle
            x1 = x + N
        
        # Check whether patch lower right coordinate exceeds image height and width
            if x1 >= width and y1 >= height:
                x1 = width - 1
                y1 = height - 1
            #tile = img[y:height, x:width]
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)
            #cv.imshow('tile', tile)
        # When patch lower right y-coordinate exceeds patch height
            elif y1 >= height:
                y1 = height - 1
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)
                
        # When patch lower right x-coordinate exceeds patch width
            elif x1 >= width:
                x1 = width - 1
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)
        
            else:
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)
            
    return img_copy

    
img = cv.imread("dog.jfif")
img2 = crop_grid(img, 2, 2, (255, 255, 255))

cv.imshow("inverse mask", img2)
cv.waitKey(0)
cv.destroyAllWindows()

In [10]:
#Question 2

img1 = cv.imread('lena.jfif')
img2 = cv.imread('coins.jfif')

# Resize img2
new_shape = img1.shape[:2]
img2 = cv.resize(img2, new_shape)

for i in np.arange(0.25, 1.00, 0.25):
    dst = cv.addWeighted(img1, i, img2, 0.25, 0)

    cv.imshow('image1', img1)
    cv.imshow('reshape_image2', img2)
    cv.imshow('blend_image', dst)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [16]:
# Question 3
#Method 1
import imutils as im

img = cv.imread("lena.jfif")
rotate = im.rotate_bound(img, 45)

cv.imshow("rotate", rotate)
cv.waitKey(0)
cv.destroyAllWindows()

In [17]:
#Method 2

import math

img = cv.imread('lena.jfif')

width = img.shape[1]
height = img.shape[0]
rangle = np.deg2rad(45)  

# calculate new image width and height
new_width = (abs(np.sin(rangle)*height) + abs(np.cos(rangle)*width))*1
new_height = (abs(np.cos(rangle)*height) + abs(np.sin(rangle)*width))*1

# calculate rotation matrix
rot_mat = cv.getRotationMatrix2D((new_width*0.5, new_height*0.5), 45, 1)

# calculate the move from the old center to the new center combined
# with the rotation
rot_move = np.dot(rot_mat, np.array([(new_width-width)*0.5, (new_height-height)*0.5,0]))

# the move only affects the translation, so just update the translation
rot_mat[0,2] += rot_move[0]
rot_mat[1,2] += rot_move[1]

dst = cv.warpAffine(img, rot_mat, (int(math.ceil(new_width)), int(math.ceil(new_height))), flags=cv.INTER_LANCZOS4)

cv.imshow('rotated_image', dst)
cv.waitKey(0)
cv.destroyAllWindows()

In [8]:
#Question 4

# Load two images
bee = cv.imread('native-bee.png')
flower = cv.imread('flower.jfif')

#create roi
roi = bee[0:flower.shape[0], 0:flower.shape[1]]

# Create mask of flower and its inverse mask 
flower_gray = cv.cvtColor(flower,cv.COLOR_BGR2GRAY)
ret, mask = cv.threshold(flower_gray, 127, 255, cv.THRESH_BINARY)
mask_inv = cv.bitwise_not(mask)

# Black out the mask of flower
bee_bg = cv.bitwise_and(roi,roi,mask = mask_inv)

# Take region of flower
flower_fg = cv.bitwise_and(flower,flower,mask = mask)

# Put flower in ROI and modify the bee image
dst = cv.add(bee_bg,flower_fg)
bee[0:flower.shape[0], 0:flower.shape[1]] = dst

cv.imshow('output',bee)
cv.waitKey(0)
cv.destroyAllWindows()